In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

print(tf.__version__)

2023-02-28 20:25:24.184593: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 20:25:24.293367: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 20:25:24.293403: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-28 20:25:24.718977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0


In [2]:
targets = [
    4.271786676,52.21072604,2110.5,1608,3.015,30.15,442.2,1708.5,201,40.2
]

In [3]:
ingredients_dict = {
    "Rind, Muskel fettfrei": [0.5, 21, 10, 160, 0.2, 3, 3, 50, 0, 1],
    # ... TODO ...
    "Eierschale": [0, 0, 33000, 0, 0, 0, 0, 0, 0, 0],
}

ingredients = pd.DataFrame(ingredients_dict, index=["Energie", "Protein", "Kalzium", "Phosphor", "Kupfer", "Zink", "Jod", "Vit. A", "Vit. D", "Vit. E"])

In [4]:
ingredients = ingredients / 100.0 # numbers entered above are per 100g, normalize to per gram
ingredients

,"Rind, Muskel fettfrei",Eierschale
Energie,0.005,0.0
Protein,0.210,0.0
Kalzium,0.100,330.0
Phosphor,1.600,0.0
Kupfer,0.002,0.0
Zink,0.030,0.0
Jod,0.030,0.0
Vit. A,0.500,0.0
Vit. D,0.000,0.0
Vit. E,0.010,0.0


In [5]:
ingredients.values

array([[5.0e-03, 0.0e+00],
       [2.1e-01, 0.0e+00],
       [1.0e-01, 3.3e+02],
       [1.6e+00, 0.0e+00],
       [2.0e-03, 0.0e+00],
       [3.0e-02, 0.0e+00],
       [3.0e-02, 0.0e+00],
       [5.0e-01, 0.0e+00],
       [0.0e+00, 0.0e+00],
       [1.0e-02, 0.0e+00]])

In [6]:
counts_shape = (ingredients.values.shape[-1], 1)

In [7]:
class Ingredients(tf.keras.layers.Layer):
    def __init__(self, ingredients, targets, **kwargs):
        super().__init__(**kwargs)
        n_values, n_ingredients = ingredients.shape
        
        self.targets = np.array(targets)
        assert self.targets.shape == (n_values,)
        
        # Normalize ingredients so they are expressed relative to their target value
        # This makes it easier to define a sensible loss later
        self.ingredients = np.array(ingredients) / self.targets.reshape((-1, 1))
    
    def build(self, input_shape=None):
        amounts_shape = (self.ingredients.shape[-1], 1)
        self.amounts = self.add_weight(
            name="amounts", shape=amounts_shape, initializer=tf.zeros,
            dtype=tf.float32
        )
        self.built = True
            
    @tf.function
    def call(self, inputs=()):
        totals = tf.convert_to_tensor(self.ingredients, dtype=tf.float32) @ self.amounts
        return totals


In [8]:
Ingredients(ingredients.values, targets).ingredients

array([[1.17047043e-03, 0.00000000e+00],
       [4.02216203e-03, 0.00000000e+00],
       [4.73821369e-05, 1.56361052e-01],
       [9.95024876e-04, 0.00000000e+00],
       [6.63349917e-04, 0.00000000e+00],
       [9.95024876e-04, 0.00000000e+00],
       [6.78426052e-05, 0.00000000e+00],
       [2.92654375e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [2.48756219e-04, 0.00000000e+00]])

In [9]:
np.ones((10,1))

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [10]:
dummy_in = tf.keras.layers.Input(batch_input_shape=())
model = tf.keras.models.Model([dummy_in], Ingredients(ingredients.values, targets)(dummy_in))

2023-02-28 20:25:25.475553: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-28 20:25:25.477899: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 20:25:25.477930: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-28 20:25:25.477952: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-28 20:25:25.477973: W tensorf

In [11]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, "MSE")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [None]                    0         
                                                                 
 ingredients_1 (Ingredients)  (10, 1)                  2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [12]:
model(0)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [31]:
for round in range(50):
    model.fit([np.array([0])], [np.ones((1,10,))], epochs=100, verbose=0)
    print(round)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [32]:
model.weights

[<tf.Variable 'ingredients_1/amounts:0' shape=(2, 1) dtype=float32, numpy=
 array([[420.0439  ],
        [  6.268168]], dtype=float32)>]

In [33]:
model([0]).numpy()[:, 0]

array([0.49164897, 1.6894846 , 0.99999994, 0.41795418, 0.2786361 ,
       0.41795418, 0.02849687, 0.1229277 , 0.        , 0.10448854],
      dtype=float32)

In [34]:
model([0]).numpy()[:, 0] * targets

array([2.10021953e+00, 8.82092174e+01, 2.11049987e+03, 6.72070316e+02,
       8.40087835e-01, 1.26013184e+01, 1.26013179e+01, 2.10021968e+02,
       0.00000000e+00, 4.20043948e+00])

In [35]:
print(["Energie", "Protein", "Kalzium", "Phosphor", "Kupfer", "Zink", "Jod", "Vit. A", "Vit. D", "Vit. E"])
print(targets)

['Energie', 'Protein', 'Kalzium', 'Phosphor', 'Kupfer', 'Zink', 'Jod', 'Vit. A', 'Vit. D', 'Vit. E']
[4.271786676, 52.21072604, 2110.5, 1608, 3.015, 30.15, 442.2, 1708.5, 201, 40.2]
